In [1]:
import requests
from pandas import DataFrame

In [ ]:
import pandas as pd
df = pd.read_csv('서울교통공사_역주소 및 전화번호_20250318(in).csv', encoding = 'utf-8')





In [56]:
import pandas as pd
import requests
import time
from urllib.parse import quote


df = pd.read_csv("서울교통공사_역주소 및 전화번호_20250318(in).csv", encoding="utf-8")

API_KEY = "0749AD16-D790-390D-A39D-78D292568419"
latitude = []
longitude = []

for i, row in df.iterrows():
    address = row["도로명주소"]
    address_encoded = quote(address)

    
    url = (
        f"https://api.vworld.kr/req/address?"
        f"service=address&request=getCoord&version=2.0&crs=EPSG:4326&"
        f"address={address_encoded}&refine=true&simple=false&format=json&type=road&key={API_KEY}"
    )

    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"[{i}] HTTP Error {response.status_code}: {response.reason}")
            x, y = None, None
        else:
            data = response.json()
            if data['response']['status'] == 'OK':
                point = data['response']['result']['point']
                x = point['x']
                y = point['y']
            else:
                print(f"[{i}] 변환 실패: {address}")
                x, y = None, None

    except Exception as e:
        print(f"[{i}] 예외 발생: {e}")
        x, y = None, None

    longitude.append(x)
    latitude.append(y)
    time.sleep(0.3)

    

df['latitude'] = latitude
df['longitude'] = longitude

df.to_csv("subway_stations_with_coords.csv", index=False, encoding="utf-8-sig")
print("모든 좌표 저장 완료➡️subway_stations_with_coords.csv 저장완료😁")


[0] 변환 실패: 서울특별시 중구 세종대로 지하2(남대문로 5가)


KeyboardInterrupt: 

In [ ]:

# 4️⃣ 주소 → 좌표 변환 루프
# ===============================
for i, row in df.iterrows():
    address = row["도로명주소"]
    address_encoded = quote(address)  # URL 인코딩

    # API 요청 URL 구성
    url = (
        f"https://api.vworld.kr/req/address?"
        f"service=address&request=getCoord&version=2.0&crs=EPSG:4326&"
        f"address={address_encoded}&refine=true&simple=false&format=json&type=road&key={API_KEY}"
    )

    try:
        # API 요청 (5초 타임아웃 설정)
        response = requests.get(url, timeout=5)

        # HTTP 오류 상태 처리
        if response.status_code != 200:
            print(f"[{i}] HTTP 오류 {response.status_code}: {response.reason}")
            x, y = None, None

        else:
            data = response.json()

            # 정상 응답일 때만 좌표 추출
            if data["response"]["status"] == "OK":
                point = data["response"]["result"]["point"]
                x, y = point["x"], point["y"]
            else:
                print(f"[{i}] 주소 변환 실패 ➜ {address}")
                x, y = None, None

    # 타임아웃 발생 시
    except requests.Timeout:
        print(f"[{i}] 요청 시간 초과 ➜ {address}")
        x, y = None, None

    # 기타 예외 (JSON 파싱 오류, 네트워크 등)
    except Exception as e:
        print(f"[{i}] 예외 발생 ({type(e).__name__}): {e}")
        x, y = None, None

    # 결과 저장
    latitude.append(y)
    longitude.append(x)

    # 서버 부하 방지를 위한 대기 (0.3초)
    time.sleep(0.3)

# ===============================
# 5️⃣ 결과를 DataFrame에 추가 후 저장
# ===============================
df["latitude"] = latitude
df["longitude"] = longitude

df.to_csv("subway_stations_with_coords.csv", index=False, encoding="utf-8")

print("✅ 모든 좌표 변환 완료 및 'subway_stations_with_coords.csv' 저장 완료 🎯")


[0] 주소 변환 실패 ➜ 서울특별시 중구 세종대로 지하2(남대문로 5가)


KeyboardInterrupt: 

In [3]:
import pandas as pd
import requests
import time
from urllib.parse import quote

In [11]:
df = pd.read_csv("서울교통공사_역주소 및 전화번호_20250318(in).csv", encoding="utf-8")
API_KEY = "0749AD16-D790-390D-A39D-78D292568419"
base_url = "https://api.vworld.kr/req/address"

service = "address"
request_type = "getCoord"
version = "2.0"
crs = "EPSG:4326"
refine = "true"
simple = "false"
format_type = "json"
addr_type = "road"

latitude = []
longitude = []
#초기화


In [13]:
for i, row in df.iterrows():
    address = row["도로명주소"]
    address_encoded = quote(address)

    query_string = (
        f"?service={service}"
        f"&request={request_type}"
        f"&version={version}"
        f"&crs={crs}"
        f"&address={address_encoded}"
        f"&refine={refine}"
        f"&simple={simple}"
        f"&format={format_type}"
        f"&type={addr_type}"
        f"&key={API_KEY}"
    )

    url = base_url + query_string

    try:
        # 5초 타임아웃 요청
        response = requests.get(url, timeout=5)

        if response.status_code == 200:
            data = response.json()

            if data["response"]["status"] == "OK":
                point = data["response"]["result"]["point"]
                x, y = point["x"], point["y"]
            else:
                print(f"[{i}] 변환 실패 ➜ {address}")
                x, y = None, None
        else:
            print(f"[{i}] HTTP 오류 {response.status_code}: {response.reason}")
            x, y = None, None

    except requests.Timeout:
        print(f"[{i}] 요청 시간 초과 ➜ {address}")
        x, y = None, None

    except Exception as e:
        print(f"[{i}] 예외 발생 ({type(e).__name__}): {e}")
        x, y = None, None

    longitude.append(x)
    latitude.append(y)

    # 진행 상황 출력
    if i % 10 == 0:
        print(f"진행률: {i}/{len(df)} ({(i/len(df))*100:.1f}%) 완료")

    time.sleep(0.3)

    df["latitude"] = latitude
    df["longitude"] = longitude

    df.to_csv("subway_stations_with_coords.csv", index=False, encoding="utf-8")

print("✅ 모든 좌표 변환 완료 및 subway_stations_with_coords.csv 저장 완료 🎯")





[0] 변환 실패 ➜ 서울특별시 중구 세종대로 지하2(남대문로 5가)
진행률: 0/289 (0.0%) 완료


ValueError: Length of values (2) does not match length of index (289)

In [59]:
import requests
from pandas import DataFrame
from concurrent import futures

In [60]:
with open("서울교통공사_역주소 및 전화번호_20250318(in).csv", "r", encoding = "utf-8") as f:
    csv_list = f.readlines()
    
print(csv_list[:5])


['\ufeff연번,역번호,호선,역명,역전화번호,도로명주소,지번주소\n', '1,150,1,서울,02-6110-1331,서울특별시 중구 세종대로 지하2(남대문로 5가),서울특별시 중구 남대문로5가 73-6 서울역(1호선)\n', '2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하101(정동),서울특별시 중구 정동 5-5 시청역(1호선)\n', '3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하55(종로1가),서울특별시 종로구 종로1가 54 종각역(1호선)\n', '4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하129(종로3가),서울특별시 종로구 종로3가 10-5 종로3가역(1호선)\n']


In [77]:
url = "https://api.vworld.kr/req/address"
params = {
	"service": "address",
	"request": "getcoord",
	"crs": "epsg:4326",
	"address": "판교로 242",
	"format": "json",
	"type": "road",
	"key": "0749AD16-D790-390D-A39D-78D292568419"
}


				

In [78]:
with requests.Session() as session:
    session.headers.update({"User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"

    })

    r = session.get(url, params = params)

    if r.status_code !=200:
        msg = "[%d Error] % 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
        


    print(r)

<Response [200]>


In [79]:
mydict = r.json()
mydict


{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'getcoord',
   'time': '24(ms)'},
  'status': 'OK',
  'input': {'type': 'road', 'address': '판교로 242'},
  'refined': {'text': '경기도 성남시 분당구 판교로 242 (삼평동)',
   'structure': {'level0': '대한민국',
    'level1': '경기도',
    'level2': '성남시 분당구',
    'level3': '삼평동',
    'level4L': '판교로',
    'level4LC': '',
    'level4A': '삼평동',
    'level4AC': '4113565500',
    'level5': '242',
    'detail': ''}},
  'result': {'crs': 'EPSG:4326',
   'point': {'x': '127.101313354', 'y': '37.402352535'}}}}

In [80]:
point = mydict['response']['result']['point']
print(point['x'], point['y'])

127.101313354 37.402352535


In [97]:
def get_point_(addr, type = "road"):

    url = "https://api.vworld.kr/req/address"
    
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": addr,
        "format": "json",
        "type": type,
        "key": "0749AD16-D790-390D-A39D-78D292568419"
    }

with requests.Session() as session:
    session.headers.update({
      "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"  
    })

    r = session.get(url, params = params)

    if r.status_code != 200:
            msg = "[%d Error] 에러 발생🚨"  % (r.status_code, r.reason)
            raise Exception(msg)
    mydict = r.json()
    point = mydict['response']['result']['point']
    return(point['x'], point['y'])

SyntaxError: 'return' outside function (3418360117.py, line 27)